In [1]:
import gymnasium as gym
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from gym import Env
import gymnasium as gym
from gymnasium import spaces
from gym.spaces import Discrete, Box
import numpy as np
import pygame
import random
import gym

In [67]:
class Platoon:
    def __init__(self, platoon_members):
        self.platoon_members = platoon_members

    def split_platoon(self, exit_index):
        # Handle last agent separating
        if exit_index == 0 and len(self.platoon_members) > 1:
            front_platoon_members = self.platoon_members[1:]  # Everything except the last agent
            exiting_agent = self.platoon_members[0]  # The last agent is the one leaving

            # Ensure the exiting agent is marked as the leader
            exiting_agent.is_leader = True
            new_exiting_platoon = Platoon([exiting_agent])
            
            # Update the original platoon to be the front platoon
            self.platoon_members = front_platoon_members
            self.platoon_members[-1].is_leader = True  # Ensure the new front platoon has a leader
            return new_exiting_platoon, None

        # Handle leader (first agent) separating
        elif exit_index == len(self.platoon_members) - 1 and len(self.platoon_members) > 1:
            rear_platoon_members = self.platoon_members[:len(self.platoon_members)-1]  # Everything except the leader
            exiting_agent = self.platoon_members[-1]  # The leader is the one leaving
            new_exiting_platoon = Platoon([exiting_agent])
            
            # Update the original platoon to be the rear platoon
            self.platoon_members = rear_platoon_members
            self.platoon_members[-1].is_leader = True  # Ensure the new rear platoon has a leader
            return new_exiting_platoon, None

        # Handle middle agent separating
        elif 0 < exit_index < len(self.platoon_members) - 1:
            rear_platoon_members = self.platoon_members[:exit_index]  # Members before the separating agent
            exiting_agent = self.platoon_members[exit_index]  # The separating agent
            front_platoon_members = self.platoon_members[exit_index + 1:]  # Members after the separating agent

            # Set up new platoons for rear and front
            exiting_agent.is_leader = True
            new_exiting_platoon = Platoon([exiting_agent])
            rear_platoon_members[-1].is_leader = True
            new_rear_platoon = Platoon(rear_platoon_members)

            # Update the original platoon to be the front platoon
            self.platoon_members = front_platoon_members
            self.platoon_members[-1].is_leader = True

            # Return both the exiting agent's new platoon and the rear platoon
            return new_exiting_platoon, new_rear_platoon

        # If there's only one agent, no split needed; return None
        return None, None

class Agent:
    def __init__(self, position, speed, destination, is_leader=False, speed_min=0.25, speed_max=1.2, acceleration=0.005):
        self.position = position
        self.speed = speed
        self.destination = destination
        self.is_leader = is_leader
        self.done = False
        self.velocities = []
        self.distances = []
        self.direction = 0
        
        # Leader-specific attributes
        self.speed_min = speed_min 
        self.speed_max = speed_max 
        self.acceleration = acceleration 
        self.target_speed = 1.0

    def update_position(self):
        self.position += self.speed

    def update_speed(self, action=None):
        if self.is_leader:
            if abs(self.speed - self.target_speed) < 0.01:
                self.speed = self.target_speed
            elif self.speed < self.target_speed:
                self.speed = min(self.speed + self.acceleration, self.target_speed)
            elif self.speed > self.target_speed:
                self.speed = max(self.speed - self.acceleration, self.target_speed)
        else:
            if action == 0:
                self.speed = max(0, self.speed - 0.005)
            elif action == 2:
                self.speed = min(1.2, self.speed + 0.05)

        self.velocities.append(self.speed)

class FollowLeaderEnv(gym.Env):
    def __init__(self, num_agents, visualize=False):
        super(FollowLeaderEnv, self).__init__()
        self.num_agents = num_agents

        # Define action and observation spaces
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(3,), dtype=np.float32)

        # Initialize agents
        initial_position = 50.0
        leader_speed = 0.0
        
        self.agents = [
            Agent(position=initial_position + 50 * (i), 
                  speed=leader_speed, 
                  destination=1 if i == num_agents - 1 else np.random.randint(2),  
                  is_leader=True if i == num_agents - 1 else False)
            for i in range(num_agents)
        ]

        # Initialize platoons with the created agents
        self.platoons = [Platoon(self.agents)]
        self.dones = {self.platoons[0]: False}

        self.target_distance = 50.0
        
        if visualize:
            self.init_pygame()

    def init_pygame(self):
        print("Initializing Pygame...")
        pygame.init()
        self.screen = pygame.display.set_mode((1000, 650))
        self.clock = pygame.time.Clock()
        self.is_pygame_initialized = True
        pygame.font.init()
        self.font = pygame.font.Font(None, 36)

    def check_distance_and_request_coupling(self, rear_platoon):
        # Loop through each platoon and find others with the same destination
        # Initialize variables to track the closest front platoon
        closest_platoon = None
        min_distance = float('inf')
        
        # Search for the closest eligible front platoon
        for front_platoon in self.platoons:
            if front_platoon is rear_platoon:
                continue  # Skip self comparison

            front_last_agent = front_platoon.platoon_members[0]
            front_first_agent = front_platoon.platoon_members[-1]
            rear_first_agent = rear_platoon.platoon_members[-1]

            # Only check platoons with the same destination and if front is ahead
            if rear_first_agent.direction == front_first_agent.direction and \
            front_last_agent.position > rear_first_agent.position:

                # Calculate the distance between rear and front platoons
                distance = front_last_agent.position - rear_first_agent.position

                # Check if this is the closest platoon within the coupling threshold
                if distance < 300 and distance < min_distance:
                    min_distance = distance
                    closest_platoon = front_platoon

        # If a closest platoon was found, perform the coupling
        if closest_platoon:
            # Adjust leadership after coupling
            rear_platoon.platoon_members[-1].is_leader = False

            # Merge the rear platoon behind the closest front platoon
            rear_platoon.platoon_members.extend(closest_platoon.platoon_members)
            rear_platoon.platoon_members[-1].is_leader = True
            # Set target speed for coupling
            rear_platoon.platoon_members[-1].target_speed = 0.5
            self.platoons.remove(closest_platoon)
            del self.dones[closest_platoon]

    def step(self, platoon, action, agent_idx):
        agent = platoon.platoon_members[agent_idx]

        if agent.destination == 1 and agent.direction == 0 and agent.position >= 500:
            agent.position = 150
            agent.direction = 1   

            # Perform the split to get two new platoons
            new_exiting_platoon, new_rear_platoon = platoon.split_platoon(agent_idx)

            # Append each platoon individually, not as a tuple
            if new_exiting_platoon:
                self.platoons.append(new_exiting_platoon)
                self.dones[new_exiting_platoon] = False
            if new_rear_platoon:
                self.platoons.append(new_rear_platoon)
                self.dones[new_rear_platoon] = False

            # Update the state and exit this step early
            return [], 0, False, {}

        leader = platoon.platoon_members[agent_idx + 1]

        agent.update_speed(action)
        agent.update_position()

        distance_to_leader = (leader.position - agent.position) 
        done = False

        if platoon.platoon_members[-1].direction == 0 and platoon.platoon_members[-1].position >= 950:
            self.dones[platoon] = True
        elif platoon.platoon_members[-1].direction == 1 and platoon.platoon_members[-1].position >= 600:
            self.dones[platoon] = True

        state = np.array([distance_to_leader, agent.speed, leader.speed], dtype=np.float32)
        agent.velocities.append(agent.speed)
        agent.distances.append(distance_to_leader)
        return state, 0, done, {}

    def update_leader_speed(self, platoon):
        any_agent_behind = any(
            (platoon.platoon_members[i + 1].position - platoon.platoon_members[i].position > self.target_distance + 10)
            for i in range(len(platoon.platoon_members) - 1)
        )

        if not any_agent_behind:
            platoon.platoon_members[-1].target_speed = 1.0 

        platoon.platoon_members[-1].update_position()
        platoon.platoon_members[-1].update_speed()

    def render(self):
        if not hasattr(self, 'screen'):
            return

        self.screen.fill((0, 0, 0))
        pygame.draw.line(self.screen, (30, 30, 30), (50, 150), (950, 150), 5)
        pygame.draw.line(self.screen, (30, 30, 30), (500, 150), (500, 600), 5)
        pygame.draw.circle(self.screen, (255, 0, 0), (950, 150), 10)
        pygame.draw.circle(self.screen, (255, 0, 0), (500, 600), 10)
        
        for platoon in self.platoons:
            for agent in platoon.platoon_members:
                if agent.direction == 0:
                    pygame.draw.rect(self.screen, (0, 255, 0) if not agent.is_leader else (0, 0, 255), pygame.Rect(int(agent.position) - 6, 147, 12, 6))
                else:
                    pygame.draw.rect(self.screen, (0, 255, 0) if not agent.is_leader else (0, 0, 255), pygame.Rect(497, int(agent.position) - 6, 6, 12))

        # speed_text = self.font.render(f"Leader Speed: {round(self.platoons[0].platoon_members[-1].speed, 3)}", True, (255, 255, 255))
        # self.screen.blit(speed_text, (10, 10))
        y_offset = 10  # Starting position for text display
        for i, platoon in enumerate(self.platoons):
            leader_speed = platoon.platoon_members[-1].speed  # Assuming last member is the leader
            text = f"Platoon {i+1} - Leader Speed: {round(leader_speed, 3)}"
            speed_text = self.font.render(text, True, (255, 255, 255))
            self.screen.blit(speed_text, (10, y_offset))
            y_offset += 20
        pygame.display.flip()
        self.clock.tick(60)

    def close(self):
        if self.is_pygame_initialized:
            print("Closing Pygame...")
            pygame.quit()
            self.is_pygame_initialized = False

In [60]:
# Neural network for Q-learning (DQN)
class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [66]:
import time

# Initialize the environment with visualization
env = FollowLeaderEnv(num_agents=6, visualize=True)  # Assuming you want 5 agents for the example

# Get the state and action dimensions from the environment
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

# Initialize a new DQN network for testing
dqn_test = DQN(state_dim, action_dim)

# Load the trained model's weights
dqn_test.load_state_dict(torch.load("policy_net_weights_final5.pth"))

# Set the model to evaluation mode 
dqn_test.eval()

# Loop to run the episode
done = False
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True

    time.sleep(0.05)

    # Iterate through each platoon, and use a while loop to handle dynamic changes in platoons
    platoon_index = 0
    while platoon_index < len(env.platoons):
        platoon = env.platoons[platoon_index]
        
        # Iterate through agents in the platoon
        agent_index = 0
        while agent_index < len(platoon.platoon_members):
            agent = platoon.platoon_members[agent_index]
            
            if agent.is_leader or agent_index == len(platoon.platoon_members) - 1:
                agent_index += 1  # Move to the next agent if current agent is the leader or last
                continue
            
            with torch.no_grad():
                next_agent = platoon.platoon_members[agent_index + 1]
                distance_to_leader = (next_agent.position - agent.position)
                
                # Decide action based on the distance to the next agent in the platoon
                if distance_to_leader > 100:
                    action = 2
                else:
                    action = torch.argmax(dqn_test(torch.FloatTensor([distance_to_leader, agent.speed, next_agent.speed]))).item()

            # Take a step in the environment
            _, _, _, _ = env.step(platoon, action, agent_index)

            if any(env.dones.values()):
                done = True
                break
            
            agent_index += 1

        # After iterating over agents, update the leader’s speed and check for coupling
        env.update_leader_speed(platoon)
        env.check_distance_and_request_coupling(platoon)
        
        # Re-check platoon list length if a split or merge occurred
        if platoon_index < len(env.platoons):
            platoon_index += 1

    env.render()

env.close()


Initializing Pygame...
Closing Pygame...


In [17]:
import matplotlib.pyplot as plt

def plot_agent_data(env):
    # Ensure no empty velocity or distance lists for agents
    valid_agents = [agent for agent in env.agents if agent.velocities and agent.distances]

    if not valid_agents:
        print("No valid agent data to plot.")
        return

    # Find the minimum length of velocities/distances across all valid agents
    min_timesteps = min(len(agent.velocities) for agent in valid_agents)
    min_timesteps = min(min_timesteps, len(env.leader.velocities))

    # Plot velocities for each agent
    plt.figure(figsize=(10, 5))
    
    # Plot velocities in one plot
    plt.subplot(1, 2, 1)
    plt.plot(range(min_timesteps), env.leader.velocities[:min_timesteps], label='Leader')
    for i, agent in enumerate(valid_agents):
        plt.plot(range(min_timesteps), agent.velocities[:min_timesteps], label=f'Agent {i+1}')
    plt.title('Velocities over Time')
    plt.xlabel('Timesteps')
    plt.ylabel('Velocity')
    plt.legend()

    # Find the minimum length of distances for all valid agents
    min_timesteps_dist = min(len(agent.distances) for agent in valid_agents)

    # Plot distances (positions) in one plot
    plt.subplot(1, 2, 2)
    for i, agent in enumerate(valid_agents):
        plt.plot(range(min_timesteps_dist), agent.distances[:min_timesteps_dist], label=f'Agent {i}')
    plt.title('Distances over Time')
    plt.xlabel('Timesteps')
    plt.ylabel('Distance')
    plt.legend()

    # Show plots
    plt.tight_layout()
    plt.show()

# Example usage:
plot_agent_data(env)



AttributeError: 'FollowLeaderEnv' object has no attribute 'leader'

Future research: decision of the leader to allow couple or not.
Sort trains by leaving order.
not slow down if non trains behind will go to other direction, knowledge of other trains.